In [1]:
import tensorflow as tf

In [2]:
x = tf.random.uniform(shape=(3,4,6,1),minval=0.,maxval=1.0)
A = tf.random.uniform(shape=(1,4,2,6),minval=0.,maxval=1.0)

Ax = tf.linalg.matmul(A,x)

In [3]:
x = tf.random.uniform(shape=(3,4,6,1),minval=0.,maxval=1.0)
A = tf.random.uniform(shape=(1,4,6,2),minval=0.,maxval=1.0)

Ax = tf.linalg.matmul(A,x,adjoint_a=True)

In [4]:
x = tf.random.uniform(shape=(5,10,10,6),minval=0.,maxval=1.0)
A = tf.random.uniform(shape=(4,4,2,6),minval=0.,maxval=1.0)

y = tf.nn.conv2d_transpose(input = x,filters = A,output_shape = x.shape[:-1] + (2,),strides = 1,padding='SAME',dilations=1)

In [5]:
import transforms as transf
sigSz = (16,16)
noc = 3
nof = 4
fltrSz = (5,5)
batchSz = 5

In [6]:
y = tf.random.uniform(shape = (batchSz,sigSz[0],sigSz[1],nof,1),minval=0,maxval=1.0,dtype = 'float64')
fft = transf.fft2d_inner(fftSz=sigSz,dtype = 'float64')
ifft = transf.ifft2d_inner(fftSz=sigSz,dtype='complex128')
D = tf.random.uniform(shape=(fltrSz[0],fltrSz[1],noc,nof),minval=0.,maxval=1.0,dtype = 'float64')
Df = fft(tf.expand_dims(D,axis = 0))
Dy = ifft(tf.matmul(Df,fft(y)))

In [7]:
Dflipped = tf.reverse(D,axis=(0,1))
#ypad1 = tf.concat([y[slice(None),slice(-tf.cast((fltrSz[0] - 1)/2,'int32') - 1,None)],y,y[slice(None),slice(tf.cast((fltrSz[0] - 1)/2 - 1,'int32'),0,-1)]],axis = 1)
ypad1 = tf.concat([y[slice(None),slice(-(fltrSz[0] - 1),None)],y],axis = 1)

#ypad2 = tf.concat([ypad1[slice(None),slice(None),slice(-tf.cast((fltrSz[1] - 1)/2 ,'int32')- 1,None)],ypad1,ypad1[slice(None),slice(None),slice(tf.cast((fltrSz[1] - 1)/2 - 1,'int32'),0,-1)]],axis = 2)
ypad2 = tf.concat([ypad1[slice(None),slice(None),slice(-(fltrSz[1] - 1),None)],ypad1],axis = 2)


outputs = tf.nn.conv2d(input = tf.reshape(ypad2,ypad2.shape[:-1]),filters = tf.transpose(Dflipped,perm=(0,1,3,2)),strides = 1,padding='VALID',dilations=1)
Dy_sp = tf.expand_dims(outputs,axis = -1)

In [8]:
#print(Dtx.shape)
#print(Dtx_sp.shape)

In [9]:
print(Dy.shape)
print(Dy_sp.shape)

#print(tf.reduce_max(tf.math.abs(Dy[slice(None),slice(fltrSz[0] - 1,None),slice(fltrSz[1] -1,None)] - Dy_sp[slice(None),slice(1,-(fltrSz[0] - 2)),slice(1,-(fltrSz[1] - 2))])))
#Dy_shift = tf.roll(tf.roll(Dy,shift = -1,axis = 1),shift = -1,axis = 2)
print(tf.reduce_max(tf.math.abs(Dy - Dy_sp)))

(5, 16, 16, 3, 1)
(5, 16, 16, 3, 1)
tf.Tensor(2.4868995751603507e-14, shape=(), dtype=float64)


In [10]:
x = tf.random.uniform(shape = (batchSz,sigSz[0],sigSz[1],noc,1),minval=0.,maxval=1.0,dtype='float64')
Dtx = ifft(tf.matmul(Df,fft(x),adjoint_a = True))


In [11]:
Dflipped = tf.reverse(D,axis=(0,1))
xpad1 = tf.concat([x,x[slice(None),slice(0,(fltrSz[0] - 1))]],axis = 1)
xpad2 = tf.concat([xpad1,xpad1[slice(None),slice(None),slice(fltrSz[1] - 1)]],axis = 2)

outputs = tf.nn.conv2d_transpose(input = tf.reshape(xpad2,xpad2.shape[:-1]),filters = tf.transpose(Dflipped,perm=(0,1,3,2)),output_shape = (batchSz,sigSz[0] + fltrSz[0] - 1,sigSz[1] + fltrSz[0] - 1,nof),strides = 1,padding='SAME',dilations=1)
Dtx_sp = tf.expand_dims(outputs,axis=-1)
#Dtx_sp = Dtx_sp[slice(None),slice(3,-3),slice(3,-3)]

In [12]:
print(Dtx_sp.shape)
print(Dtx.shape)
temp_int = [tf.cast((fltrSz[ii] - 1)/2,'int32') for ii in range(2)]
print(tf.reduce_max(tf.math.abs(Dtx_sp[slice(None),slice(temp_int[0],-temp_int[0]),slice(temp_int[1],-temp_int[1])] - Dtx)))

(5, 20, 20, 4, 1)
(5, 16, 16, 4, 1)
tf.Tensor(2.1316282072803006e-14, shape=(), dtype=float64)


In [13]:
print(tf.reshape(Dtx[slice(0,1),slice(-6,None),slice(-6,None),slice(0,1)],shape = (6,6)))

tf.Tensor(
[[15.99123056 17.05666027 16.91037214 16.25444076 16.85907998 14.24533559]
 [15.52074591 16.55530125 17.74946339 18.04954743 15.43543663 16.67775218]
 [14.46792519 16.364748   15.33314368 14.82934257 16.56783947 14.14178086]
 [14.70711021 14.14508482 15.56960533 16.39970273 17.27085034 16.01211043]
 [15.30412675 15.20891187 14.75264049 16.89282141 15.79365292 17.37518686]
 [13.95527863 13.55722032 13.5090681  14.06598006 15.2152124  15.85730997]], shape=(6, 6), dtype=float64)


In [14]:
print(tf.reshape(Dtx_sp[slice(0,1),slice(-9,-3),slice(-9,-3),slice(0,1)],shape = (6,6)))

tf.Tensor(
[[15.00790031 16.31499405 15.22879988 14.18725121 14.6427946  12.43583261]
 [15.93779338 15.99123056 17.05666027 16.91037214 16.25444076 16.85907998]
 [15.28226267 15.52074591 16.55530125 17.74946339 18.04954743 15.43543663]
 [14.74881657 14.46792519 16.364748   15.33314368 14.82934257 16.56783947]
 [14.26384868 14.70711021 14.14508482 15.56960533 16.39970273 17.27085034]
 [15.45720295 15.30412675 15.20891187 14.75264049 16.89282141 15.79365292]], shape=(6, 6), dtype=float64)


In [15]:
#print(tf.reduce_max(tf.math.abs(Dtx - Dtx_sp)))

Dy is a shifted version of Dy_sp.

In [16]:
#print(tf.reshape(Dy[slice(0,1),slice(None),slice(None),slice(0,1)],shape = (6,6)))

In [17]:
#print(tf.reshape(Dy_sp[slice(0,1),slice(None),slice(None),slice(0,1)],shape = (6,6)))

In [18]:
import matrix_decompositions_tf as fctr

print(Df.shape)

(1, 16, 9, 3, 4)


In [19]:
divide_by_R = fctr.Coef_Divide_By_R(tf.expand_dims(D,axis=0),noc = D.shape[2])

In [20]:
dhmul = fctr.DhMul(fft(divide_by_R.D),dtype = tf.complex128)

In [21]:
dmul = fctr.DMul(dhmul,dtype = tf.complex128)

In [22]:
print(x.shape)

(5, 16, 16, 3, 1)


In [23]:
Dhx = ifft(dhmul(fft(x)))

In [24]:
Dy = ifft(dmul(fft(y)))

In [25]:
print(Dhx.shape)

(5, 16, 16, 4, 1)


In [26]:
print(Dy.shape)

(5, 16, 16, 3, 1)


In [27]:
Dconv = fctr.MulD_Sp(divide_by_R,dtype = 'float64')

In [28]:
Dtconv = fctr.MulDT_Sp(divide_by_R,dtype = 'float64')

In [29]:
print(x.shape)

(5, 16, 16, 3, 1)


In [30]:
Dhx_sp = Dtconv(tf.transpose(x,perm = (0,1,2,4,3)))

InvalidArgumentError: Input to reshape is a tensor with 4800 values, but the requested shape has 1600 [Op:Reshape]

In [31]:
Dy_sp = Dconv(tf.transpose(y,perm = (0,1,2,4,3)))

InvalidArgumentError: Conv2DCustomBackpropInput: input depth must be evenly divisible by filter depth [Op:Conv2DBackpropInput]